# 1.Importação das libs

In [15]:
import pandas as pd
import psycopg2 as pg
import panel as pn


In [16]:
conn = pg.connect(
    host="localhost",
    database="testp",
    user="postgres",
    password="jabuticaba12",
)

In [17]:
cursor = conn.cursor()

In [18]:
pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'5feafba4-93af-48ee-9fe7-8f98da88a503': {'version…

# 2.Carregamento da Tabela

In [19]:
cursor.execute("SELECT * FROM vaga")
result = cursor.fetchall()  
df=pd.DataFrame(result, columns=['id' ,'id_empresa','titulo' ,'descricao','requisitos' ,'salario','beneficios','adaptacoes' ,'deficiencias_atendidas'])
df_widget = pn.widgets.Tabulator(df, buttons={'Print':"<i class='fa fa-print'></i>"})

# 3.Widgets da Busca

In [20]:
flag=''
titulo= pn.widgets.TextInput(name='titulo', placeholder='Digite a função que deseja exercer')
buttonPesquisar=pn.widgets.Button(name='Pesquisar', button_type='primary')
salarioslider = pn.widgets.IntSlider(name='Salario', start=0, end=15000, step=100, value=0)



# 4.Função para pesquisar vagas(READ)

In [21]:
def on_pesquisar(event):
    try:
        conn.rollback()
        query = """SELECT * FROM vaga WHERE ((%s = %s OR titulo ILIKE %s ) AND salario <= %s)"""
        params = (titulo.value, flag, f"%{titulo.value}%", salarioslider.value)  
        cursor.execute(query, params)
        result_p = cursor.fetchall()
        df_p = pd.DataFrame(result_p, columns=['id', 'id_empresa', 'titulo', 'descricao', 'requisitos', 'salario', 'beneficios', 'adaptacoes', 'deficiencias_atendidas'])
        df_widget.value = df_p

    except Exception as e:
        conn.rollback()
        pn.state.notifications.error(f'Erro ao pesquisar: {str(e)}')

# 5.Widgets da Criação, Atualizão e Remoção(Create, Update e Read)

## 5.1 Wigets de entrada

In [22]:
id = pn.widgets.NumberInput(name='ID:',start=0, step=1, value=0)
id_empresa= pn.widgets.NumberInput(name='ID da Empresa:',start=0, step=1, value=0)
titulo_c= pn.widgets.TextInput(name='Titulo:', placeholder='Digite o titulo da vaga')
descricao= pn.widgets.TextAreaInput(name='Descrição:', placeholder='Digite a descrição da vaga')
requisitos= pn.widgets.TextInput(name='Requisitos:', placeholder='Digite os requisitos da vaga')
salario= pn.widgets.NumberInput(name='Salário:',start=0, step=100, value=0)
beneficios= pn.widgets.TextAreaInput(name='Benefícios:', placeholder='Digite os beneficios da vaga')
adaptacoes= pn.widgets.TextInput(name='Adaptações:', placeholder='Digite as adaptacoes da vaga')
deficiencias_atendidas= pn.widgets.MultiChoice(name='Deficiências Atendidas:', options=['Física', 'Visual', 'Auditiva', 'Intelectual', 'Múltipla', 'Autismo'] )
id_remove= pn.widgets.NumberInput(name='ID:',start=0, step=1, value=0)


## 5.1 Botões

In [23]:
buttonCriar=pn.widgets.Button(name='Criar ou Atualizar', button_type='primary')
buttonRemover=pn.widgets.Button(name='Deletar', button_type='danger')

In [24]:
def on_criar(event):
    try:
        conn.rollback()
        if not titulo_c.value or not id_empresa.value or not salario.value or not deficiencias_atendidas.value:
            pn.state.notifications.error('Preencha todos os campos obrigatórios')
            return
    
        deficiencias_atendidas_c = ','.join(deficiencias_atendidas.value)

        if id.value:
            cursor.execute("SELECT * FROM vaga WHERE id = %s", (id.value,))
            if not cursor.fetchone():
                pn.state.notifications.error('Vaga não encontrada')
                return
            uptade_query = """UPDATE vaga SET id_empresa = %s, titulo = %s, descricao = %s, requisitos = %s, salario = %s, beneficios = %s, adaptacoes = %s, deficiencias_atendidas = %s WHERE id = %s"""
            params = (id_empresa.value, titulo_c.value, descricao.value or None, requisitos.value or None, salario.value, beneficios.value or None, adaptacoes.value or None, deficiencias_atendidas_c, id.value)
            cursor.execute(uptade_query, params)
            conn.commit()
            pn.state.notifications.success('Vaga atualizada com sucesso')

        else:
            cursor.execute("SELECT MAX(ID) FROM vaga")
            max_id = cursor.fetchone()[0]
            new_id = max_id + 1 if max_id else 1
            insert_query = """INSERT INTO vaga (id, id_empresa, titulo, descricao, requisitos, salario, beneficios, adaptacoes, deficiencias_atendidas) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            params = (new_id, id_empresa.value, titulo_c.value, descricao.value or None, requisitos.value or None, salario.value, beneficios.value or None, adaptacoes.value or None, deficiencias_atendidas_c)
            cursor.execute(insert_query, params)
            conn.commit()
            pn.state.notifications.success(f"Vaga criada com sucesso, ID gerado: {new_id}")
        
        cursor.execute("SELECT * FROM vaga")
        result = cursor.fetchall()
        df_uptade=pd.DataFrame(result, columns=['id' ,'id_empresa','titulo' ,'descricao','requisitos' ,'salario','beneficios','adaptacoes' ,'deficiencias_atendidas'])
        df_widget.value = df_uptade

        id.value = ''
        id_empresa.value = 0
        titulo_c.value = ''
        descricao.value = ''
        requisitos.value = ''
        salario.value = 0
        beneficios.value = ''
        adaptacoes.value = ''
        deficiencias_atendidas_c.value = []

    except Exception as e:
        conn.rollback()
        pn.state.notifications.error(f'Erro ao criar vaga: {str(e)}')

        
    

# 6. Função de Remover

In [25]:
def on_remover(event):
     try:
          conn.rollback()
          if not id_remove.value:
               pn.state.notifications.error('ID é obrigatório.')
               return
          
          cursor.execute("SELECT * FROM vaga WHERE id = %s", (id_remove.value,))
          if not cursor.fetchone():
               pn.state.notifications.error('Vaga não encontrada.')
               return
          
          remove_query = """DELETE FROM vaga WHERE id = %s"""
          params = (id_remove.value,)
          cursor.execute(remove_query, params)
          conn.commit()
          pn.state.notifications.success('Vaga removida com sucesso!')

          cursor.execute("SELECT * FROM vaga")
          result = cursor.fetchall()
          df_update = pd.DataFrame(result, columns=['id' ,'id_empresa','titulo' ,'descricao','requisitos' ,'salario','beneficios','adaptacoes' ,'deficiencias_atendidas'])
          df_widget.value = df_update

          id_remove.value = 0

     except Exception as e:
          conn.rollback()
          pn.state.notifications.error(f'Erro ao remover vaga: {str(e)}')


# 7.Binding dos Botões

In [26]:
buttonPesquisar.on_click(on_pesquisar)
buttonCriar.on_click(on_criar)
buttonRemover.on_click(on_remover)


Watcher(inst=Button(button_type='danger', name='Deletar'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_remover at 0x00000179439CAA20>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

# 8.Layout

In [27]:
layout= pn.Column(
    "# Consulta de Candidatos",
    titulo,
    salarioslider,
    buttonPesquisar,
    df_widget,
    pn.layout.Divider(),
    pn.Row(
        pn.Column(
            "# Cadastro de Vagas",
            id,
            id_empresa,
            titulo_c,
            descricao,
            requisitos,
            salario,
            beneficios,
            adaptacoes,
            deficiencias_atendidas,
            buttonCriar
        ),
        pn.Column(
            "# Deletar Vaga",
            id_remove,
            buttonRemover
        )
    )
)
    

In [28]:
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'3ae6e716-a943-4cf2-acd4-42a9ea7b8cfb': {'version…